In [1]:
import logging
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import argparse
import os
import random
import numpy as np
from torch.autograd import Variable
from torch.utils.data import DataLoader
import utils
import itertools
from tqdm import tqdm_notebook
import models.dcgan_unet_64 as dcgan_unet_models
import models.dcgan_64 as dcgan_models
import models.classifiers as classifiers
import models.my_model as my_model
from data.moving_mnist import MovingMNIST

In [2]:
torch.cuda.set_device(1)

Constant definition

In [3]:
np.random.seed(1)
random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
lr = 2e-3
seq_len = 12
beta1 = 0.5
content_dim = 128
pose_dim = 50
channels = 3
normalize = False
sd_nf = 100
image_width = 64
batch_size = 100
log_dir = './logs/0529_noiseLikePoseVector/'
os.makedirs(os.path.join(log_dir, 'rec'), exist_ok=True)
os.makedirs(os.path.join(log_dir, 'analogy'), exist_ok=True)
logging.basicConfig(filename=os.path.join(log_dir, 'record.txt'), level=logging.DEBUG)

Data Loader

In [5]:
train_data = MovingMNIST(True, '../data_uni/', seq_len=seq_len)
test_data = MovingMNIST(False, '../data_uni/', seq_len=seq_len)

train_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    num_workers=16,
    shuffle=True,
    drop_last=True,
    pin_memory=True
)
test_loader = DataLoader(
    test_data,
    batch_size=batch_size,
    num_workers=0,
    shuffle=True,
    drop_last=True,
    pin_memory=True
)

Model definition

In [6]:
# # netEC = dcgan_unet_models.content_encoder(content_dim, channels).to(device)
# netEC = dcgan_models.content_encoder(content_dim, channels).to(device)
# netEP = dcgan_models.pose_encoder(pose_dim, channels).to(device)
# # netD = dcgan_unet_models.decoder(content_dim, pose_dim, channels).to(device)
# netD = dcgan_models.decoder(content_dim, pose_dim, channels).to(device)
# netC = classifiers.scene_discriminator(pose_dim, sd_nf).to(device)

netEC = my_model.content_encoder(content_dim, channels).to(device)
netEP = my_model.pose_encoder(pose_dim, channels, conditional=True).to(device)
netD = my_model.decoder(content_dim, pose_dim, channels).to(device)
# netC = my_model.scene_discriminator(pose_dim, sd_nf).to(device)
# netC = my_model.Discriminator(channels).to(device)

netEC.apply(utils.weights_init)
netEP.apply(utils.weights_init)
netD.apply(utils.weights_init)
# netC.apply(utils.weights_init)

print(netEC)
print(netEP)
print(netD)
# print(netC)

content_encoder(
  (main): Sequential(
    (0): dcgan_conv(
      (main): Sequential(
        (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
      )
    )
    (1): dcgan_conv(
      (main): Sequential(
        (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
      )
    )
    (2): dcgan_conv(
      (main): Sequential(
        (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
      )
    )
    (3): dcgan_conv(
      (main): Sequential(
        (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
 

In [7]:
optimizerEC = optim.Adam(netEC.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerEP = optim.Adam(netEP.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
# optimizerC = optim.Adam(netC.parameters(), lr=lr, betas=(beta1, 0.999))

Plot function

In [8]:
# --------- plotting funtions ------------------------------------
def plot_rec(x, epoch, dtype):
    x_c = x[0]
    x_p = x[np.random.randint(1, len(x))]

    h_c = netEC(x_c)
    h_p = netEP(x_p, h_c)
    rec = netD([h_c, h_p])

    x_c, x_p, rec = x_c.data, x_p.data, rec.data
    fname = '{}-{}.png'.format(dtype, epoch)
    fname = os.path.join(log_dir, 'rec', fname)
    to_plot = []
    row_sz = 5
    nplot = 20
    for i in range(0, nplot-row_sz, row_sz):
        row = [[xc, xp, xr] for xc, xp, xr in zip(x_c[i:i+row_sz], x_p[i:i+row_sz], rec[i:i+row_sz])]
        to_plot.append(list(itertools.chain(*row)))
    utils.save_tensors_image(fname, to_plot)

def plot_analogy(x, epoch, dtype):
    x_c = x[0]
    h_c = netEC(x_c)
    
    nrow = 10
    row_sz = len(x)
    to_plot = []
    row = [xi[0].data for xi in x]
    zeros = torch.zeros(channels, image_width, image_width)
    to_plot.append([zeros] + row)
    for i in range(nrow):
        to_plot.append([x[0][i].data])

    for j in range(0, row_sz):
        # for each time step
        h_p = netEP(x[j], h_c).data
        # first 10 pose vector, equal to first pose vector
        for i in range(nrow):
            h_p[i] = h_p[0]
        rec = netD([h_c, h_p])
        for i in range(nrow):
            to_plot[i+1].append(rec[i].data.clone())

    fname = '{}-{}.png'.format(dtype, epoch)
    fname = os.path.join(log_dir, 'analogy', fname)
    utils.save_tensors_image(fname, to_plot)

Training function

In [9]:
def train(x):
    optimizerEC.zero_grad()
    optimizerEP.zero_grad()
    optimizerD.zero_grad()

    x_c1 = x[np.random.randint(len(x))]
    x_c2 = x[np.random.randint(len(x))]
    x_p = x[np.random.randint(len(x))]

    h_c1 = netEC(x_c1)
    h_c2 = netEC(x_c2).detach()
    h_p = torch.randn((h_c1.size(0), pose_dim, 1, 1), device=device)


    # similarity loss: ||h_c1 - h_c2||
    sim_loss = F.mse_loss(h_c1, h_c2)


    # reconstruction loss: ||D(h_c1, h_p1), x_p1|| 
    rec = netD([h_c1, h_p])
    rec_loss = F.mse_loss(rec, x_p)
    
    # Noise reconstruction loss
    h_p_rec = netEP(rec.detach(), h_c1)
    noise_loss = F.mse_loss(h_p_rec, h_p)
    
    # full loss
    loss = sim_loss + rec_loss + 0.1 * noise_loss
    loss.backward()

    optimizerEC.step()
    optimizerEP.step()
    optimizerD.step()

    return sim_loss.item(), rec_loss.item(), noise_loss.item()

In [10]:
def train_scene_discriminator(x):
    optimizerC.zero_grad()

    target = torch.FloatTensor(batch_size, 1).to(device)
    
    # condition
    h_c = netEC(x[np.random.randint(len(x))]).detach()
    
    x1 = x[0]
    x2 = x[1]
    h_p1 = netEP(x1, h_c).detach()
    h_p2 = netEP(x2, h_c).detach()

    half = batch_size // 2
    rp = torch.randperm(half).cuda()
    h_p2[:half] = h_p2[rp]
    target[:half] = 0
    target[half:] = 1

    out = netC([h_p1, h_p2])
    bce = F.binary_cross_entropy(out, target)

    bce.backward()
    optimizerC.step()

    acc =out[:half].le(0.5).sum() + out[half:].gt(0.5).sum()
    return bce.data.cpu().numpy(), acc.data.cpu().numpy() / batch_size

In [11]:
def train_discriminator(x):
    optimizerC.zero_grad()

    real_lbl = torch.FloatTensor(batch_size, 1).fill_(1).to(device)
    fake_lbl = torch.FloatTensor(batch_size, 1).fill_(0).to(device)
    
    x1 = x[np.random.randint(len(x))]
    x2 = x[np.random.randint(len(x))]
    x3 = x[np.random.randint(len(x))]

    # real pair
    # 1. x1
    # 2. reconstructed frames by pose(x2) and content(x1)
    h_c = netEC(x1).detach()
    h_p = netEP(x3, h_c)
    x_rec = netD([h_c, h_p]).detach()
    out_real = netC([x1, x_rec]).view(-1, 1)
    loss_real = F.binary_cross_entropy(out_real, real_lbl)
    D_real = loss_real.mean().item()
    
    # fake pair
    # 1. x1
    # 2. swapped reconstructed frames
    #    by swapped pose(x3) and content(x1)
    idx = torch.randperm(batch_size)
    h_p = netEP(x3, h_c)
    h_p = h_p[idx]
    x_swap = netD([h_c, h_p]).detach()
    out_fake = netC([x1, x_swap]).view(-1, 1)
    loss_fake = F.binary_cross_entropy(out_fake, fake_lbl)
    D_fake = loss_fake.mean().item()
    
    
    bce = 0.5*loss_real + 0.5*loss_fake
    bce.backward()
    optimizerC.step()

    
    return bce.item(), D_real, D_fake

In [12]:
epoch_size = len(train_loader)
test_x = next(iter(test_loader))
test_x = torch.transpose(test_x, 0, 1)
test_x = test_x.to(device)

for epoch in tqdm_notebook(range(200), desc='EPOCH'):
    netEP.train()
    netEC.train()
    netD.train()
#     netC.train()
    epoch_sim_loss, epoch_rec_loss, epoch_noise_loss, epoch_sd_loss = 0, 0, 0, 0
    epoch_D_real, epoch_D_fake, epoch_D_G_fake = 0, 0, 0
    
    for i, x in enumerate(tqdm_notebook(train_loader, desc='BATCH')):
        # x to device
        x = torch.transpose(x, 0, 1)
        x = x.to(device)
        
        # train scene discriminator
        # sd_loss, sd_acc = train_scene_discriminator(x)
#         sd_loss, D_real, D_fake = train_discriminator(x)
#         epoch_sd_loss += sd_loss
#         epoch_D_real += D_real
#         epoch_D_fake += D_fake
        
        # train main model
        sim_loss, rec_loss, noise_loss = train(x)
        epoch_sim_loss += sim_loss
        epoch_rec_loss += rec_loss
        epoch_noise_loss += noise_loss
#         epoch_adv_loss += adv_loss
#         epoch_D_G_fake += D_G_fake
    
    log_str='[%02d]rec loss: %.4f |sim loss: %.4f|noise loss: %.4f' %\
    (epoch,
     epoch_rec_loss/epoch_size,
     epoch_sim_loss/epoch_size,
     epoch_noise_loss/epoch_size
     )
    
    print(log_str)
    logging.info(log_str)
    
    netEP.eval()
    netEC.eval()
    netD.eval()
    
    with torch.no_grad():
        plot_rec(test_x, epoch, 'test')
        plot_analogy(test_x, epoch, 'test')

    # save the model
    torch.save({
        'netD': netD,
        'netEP': netEP,
        'netEC': netEC},
        '%s/model.pth' % log_dir)

[00]rec loss: 0.0159 |sim loss: 0.0354|noise loss: 1.2883


[01]rec loss: 0.0140 |sim loss: 0.0067|noise loss: 0.9991


[02]rec loss: 0.0140 |sim loss: 0.0030|noise loss: 0.9715


[03]rec loss: 0.0139 |sim loss: 0.0015|noise loss: 0.9492


[04]rec loss: 0.0139 |sim loss: 0.0011|noise loss: 0.9326


...

[195]rec loss: 0.0128 |sim loss: 0.0001|noise loss: 0.7031


[196]rec loss: 0.0127 |sim loss: 0.0001|noise loss: 0.7101


[197]rec loss: 0.0128 |sim loss: 0.0001|noise loss: 0.7188


[198]rec loss: 0.0128 |sim loss: 0.0001|noise loss: 0.7102


[199]rec loss: 0.0128 |sim loss: 0.0001|noise loss: 0.7139



In [ ]:
len(train_loader)

In [ ]:
for i, x in enumerate(train_loader):
    if i == 0:
        with torch.no_grad():
            x = torch.transpose(x, 0, 1)
            x = x.to(device)
            plot_rec(x, 200)
            plot_analogy(x, 200)